<a href="https://colab.research.google.com/github/junieberry/DL-fromScratch/blob/main/13_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd /content/drive/MyDrive/밑시딥/deep-learning-from-scratch-2

/content/drive/MyDrive/밑시딥/deep-learning-from-scratch-2


### Preprocess

In [ ]:
import sys
sys.path.append('..')
from common.util import preprocess
import numpy as np


text="I like her and she likes him."
corpus, word_to_id,id_to_word=preprocess(text)

print(corpus)
print(word_to_id)
print(id_to_word)

[0 1 2 3 4 5 6 7]
{'i': 0, 'like': 1, 'her': 2, 'and': 3, 'she': 4, 'likes': 5, 'him': 6, '.': 7}
{0: 'i', 1: 'like', 2: 'her', 3: 'and', 4: 'she', 5: 'likes', 6: 'him', 7: '.'}


### Context

In [ ]:
def create_contexts_target(corpus, window_size=1):
  target=corpus[window_size:-window_size]
  contexts=[]

  for idx in range(window_size, len(corpus)-window_size):
    cs=[]
    for t in range(-window_size,window_size+1):
      if t==0:
        continue
      cs.append(corpus[idx+t])
    contexts.append(cs)

  return np.array(contexts),np.array(target) 

In [ ]:
contexts, target=create_contexts_target(corpus, window_size=1)
vocab_size=len(word_to_id)
target=convert_one_hot(target,vocab_size)
contexts=convert_one_hot(contexts, vocab_size)
print(contexts)
print(target)

[[[1 0 0 0 0 0 0 0]
  [0 0 1 0 0 0 0 0]]

 [[0 1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0]]

 [[0 0 1 0 0 0 0 0]
  [0 0 0 0 1 0 0 0]]

 [[0 0 0 1 0 0 0 0]
  [0 0 0 0 0 1 0 0]]

 [[0 0 0 0 1 0 0 0]
  [0 0 0 0 0 0 1 0]]

 [[0 0 0 0 0 1 0 0]
  [0 0 0 0 0 0 0 1]]]
[[0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]]


### CBOW

In [ ]:
from common.layers import MatMul, SoftmaxWithLoss

class SimpleCBOW:
  def __init__(self, vocab_size, hidden_size):
    V, H = vocab_size, hidden_size

    W_in=0.01*np.random.rand(V,H).astype('f')
    W_out=0.01*np.random.rand(H,H).astype('f')

    self.in_layer0=MatMul(W_in)
    self.in_layer1=MatMul(W_in)
    self.out_layer=MatMul(W_out)
    self.loss_layer=SoftmaxWithLoss()

    layers=[self.in_layer0, self.in_layer1, self.out_layer]
    self.params, self.grads=[],[]
    for layer in layers:
      self.params+=layer.params
      self.grads+=layer.grads

    self.word_vecs=W_in
  
  def forward(self, contexts, target):
    h0=self.in_layer0.forward(contexts[:,0])
    h1=self.in_layer1.forward(contexts[:,1])
    h=(h0+h1)%0.5
    score=self.out_layer.forward(h)
    loss=self.loss_layer.forward(score,target)
    return loss

  def backward(self, dout=1):
    ds=self.loss_layer.backward(dout)
    da=self.out_layer.backward(ds)
    da*=0.5
    self.in_layer1.backward(da)
    self.in_layer0.backward(da)
    return None

In [ ]:
from common.trainer import Trainer
from common.optimizer import Adam
from common.util import convert_one_hot

window_size=1
hidden_size=5
batch_size=3
max_epoch=1000


model=SimpleCBOW(vocab_size, hidden_size)

optimizer=Adam()
trainer=Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()


IndexError: ignored

In [ ]:
word_vecs = model.word_vecs
for word_id, word in id_to_word.items():
  print(word, word_vecs[word_id])

i [0.00864397 0.00121321 0.00242075 0.00237929 0.00776211]
like [0.00103096 0.0069158  0.00533361 0.0077615  0.00881737]
you [0.00416769 0.00180885 0.00288144 0.00827478 0.00796892]
him [0.00316207 0.00685497 0.00799248 0.00354544 0.00416911]
